# SQL Queries (Part II)

##### E-R Diagram

![E-R Diagram](soccer.drawio.png "E-R Diagram")

In [2]:
!wget -c https://github.com/bdist/is-labs-data/raw/refs/heads/main/soccer/database.sqlite?download= -O /home/jovyan/data/database.sqlite

--2025-08-07 16:17:07--  https://github.com/bdist/is-labs-data/raw/refs/heads/main/soccer/database.sqlite?download=
Resolving github.com (github.com)... 140.82.121.4
connected. to github.com (github.com)|140.82.121.4|:443... 
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/bdist/is-data/raw/refs/heads/main/soccer/database.sqlite?download= [following]
--2025-08-07 16:17:08--  https://github.com/bdist/is-data/raw/refs/heads/main/soccer/database.sqlite?download=
Reusing existing connection to github.com:443.
302 Foundest sent, awaiting response... 
Location: https://media.githubusercontent.com/media/bdist/is-data/refs/heads/main/soccer/database.sqlite?download=true [following]
--2025-08-07 16:17:08--  https://media.githubusercontent.com/media/bdist/is-data/refs/heads/main/soccer/database.sqlite?download=true
185.199.108.133, 185.199.109.133, 185.199.110.133, ...rcontent.com)... 
Connecting to media.githubusercontent.com (media.githubusercontent.c

In [3]:
%reload_ext sql
%config SqlMagic.displaycon = 0
%config SqlMagic.displaylimit = 100
%config SqlMagic.feedback = 0
%sql sqlite:////home/jovyan/data/database.sqlite --alias sqlite

## Queries

1. What is the maximum number of goals recorded in a match? Output a single number.

In [4]:
%%sql sqlite
SELECT
  MAX(home_team_goal + away_team_goal) AS max_total_match_goals
FROM
  MATCH;

max_total_match_goals
12


2. List each league name and its number of teams sorted from high to low.

In [6]:
%%sql sqlite
SELECT
  b.name,
  COUNT(*) AS count_teams
FROM
  (
    SELECT DISTINCT
      home_team_api_id,
      league_id
    FROM
      MATCH
  ) a
  LEFT JOIN league AS b ON a.league_id = b.id
GROUP BY
  league_id
ORDER BY
  count_teams DESC;

name,count_teams
France Ligue 1,35
England Premier League,34
Spain LIGA BBVA,33
Italy Serie A,32
Germany 1. Bundesliga,30
Portugal Liga ZON Sagres,29
Netherlands Eredivisie,25
Belgium Jupiler League,24
Poland Ekstraklasa,22
Scotland Premier League,17


3. Who are the GoalKeepers (GK) that have played all the away games for their team? Output the `team_long_name` and `player_name` (assuming `match.away_player_1` is the goalkeeper column).

In [7]:
%%sql sqlite
SELECT
  team_long_name,
  player_name
FROM
  (
    SELECT
      away_team_api_id,
      COUNT(*) AS total_team_away_matches,
      away_player_1
    FROM
      MATCH
    GROUP BY
      away_team_api_id
  ) a
  LEFT JOIN (
    SELECT
      away_player_1,
      COUNT(*) AS total_gk_matches
    FROM
      MATCH
    GROUP BY
      away_player_1
  ) b ON a.away_player_1 = b.away_player_1
  LEFT JOIN player AS c ON b.away_player_1 = c.player_api_id
  LEFT JOIN team AS d ON a.away_team_api_id = d.team_api_id
WHERE
  total_team_away_matches = total_gk_matches;

team_long_name,player_name
Angers SCO,Ludovic Butelle
SV Darmstadt 98,Christian Mathenia
SC Paderborn 07,Lukas Kruse
Norwich City,John Ruddy
CD Tenerife,Sergio Aragones
DSC Arminia Bielefeld,Dennis Eilhoff
Hércules Club de Fútbol,Juan Jesus Calatayud


## PART II - New Database Schema (PostgreSQL)

#### 0. Migrate the SQLite Database to PostgreSQL

Instead of writing tedious migration code ourselves, we will be using the `pgloader` CLI tool to get started quickly. The migration task is setup using a `command-file`. Lets see inside that file.

In [10]:
!cat ./command-file.txt

load database
    from '/home/jovyan/data/database.sqlite'
    into postgresql://db:db@postgres/db

with include drop, create tables, create indexes, reset sequences,

prefetch rows = 100, max parallel create index = 1

set work_mem to '16MB', maintenance_work_mem to '512 MB';


In [12]:
!pgloader ./command-file.txt

2025-08-07T16:37:55.020648Z LOG pgloader version "3.6.10~devel"
2025-08-07T16:37:55.020648Z LOG Data errors in '/tmp/pgloader/'
2025-08-07T16:37:55.020648Z LOG Parsing commands from file #P"/home/jovyan/work/command-file.txt"
2025-08-07T16:37:55.196154Z LOG Migrating from #<SQLITE-CONNECTION sqlite:///home/jovyan/data/database.sqlite {1006A3D4E3}>
2025-08-07T16:37:55.206478Z LOG Migrating into #<PGSQL-CONNECTION pgsql://db@postgres:5432/db {1006A3D663}>
2025-08-07T16:38:21.591172Z LOG report summary reset
             table name     errors       rows      bytes      total time
-----------------------  ---------  ---------  ---------  --------------
                  fetch          0          0                     0.000s
        fetch meta data          0         50                     0.124s
         Create Schemas          0          0                     0.000s
       Create SQL Types          0          0                     0.021s
          Create tables          0         14      

In [13]:
%load_ext sql
%config SqlMagic.displaycon = 0
%config SqlMagic.displaylimit = 100
%config SqlMagic.feedback = 0
%sql postgresql+psycopg://db:db@postgres/db --alias psql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [14]:
%sqlcmd columns -t match

name,type,nullable,default,autoincrement,comment
id,BIGINT,False,nextval('match_id_seq'::regclass),True,None
country_id,BIGINT,True,None,False,None
league_id,BIGINT,True,None,False,None
season,TEXT,True,None,False,None
stage,BIGINT,True,None,False,None
date,TEXT,True,None,False,None
match_api_id,BIGINT,True,None,False,None
home_team_api_id,BIGINT,True,None,False,None
away_team_api_id,BIGINT,True,None,False,None
home_team_goal,BIGINT,True,None,False,None


#### 1. Functions, Stored Procedures, and Triggers

Provide the SQL instructions corresponding to each of the following tasks:

1. Create a new `match_player` table to store the data for the starting players (11) for each match (currently stored in the columns `match.home_player_DD` and `match.away_player_DD`). We are going to copy the data contained in the table `match` to `match_player` but we want to use a new column for the player position `position`.

In [15]:
%%sql psql
DROP TABLE IF EXISTS match_player;

CREATE TABLE
  match_player (
    id SERIAL,
    player_id BIGINT,
    team_id BIGINT,
    match_id BIGINT,
    league_id BIGINT,
    POSITION BIGINT,
    cond TEXT,
    PRIMARY KEY (id),
    FOREIGN KEY (player_id) REFERENCES player (player_api_id),
    FOREIGN KEY (match_id) REFERENCES MATCH (id),
    FOREIGN KEY (team_id) REFERENCES team (team_api_id),
    FOREIGN KEY (league_id) REFERENCES league (id)
  );

++
||
++
++

2. Populate the new table `match_player` using the data contained in the table `match` using a function or stored procedure.

In [16]:
%%sql psql

DELETE FROM match_player;

CREATE OR REPLACE FUNCTION populate_match_player()
RETURNS void AS $$
DECLARE
    match_row RECORD;
    i INT;
    home_player_id INT;
    away_player_id INT;
BEGIN
    FOR match_row IN SELECT * FROM match LOOP
        FOR i IN 1..11 LOOP
            EXECUTE format(
                'SELECT %I FROM match WHERE id = %L',
                format('home_player_%s', i),
                match_row.id
            ) INTO home_player_id;

            IF home_player_id IS NOT NULL THEN
                INSERT INTO match_player (player_id, team_id, league_id, match_id, position, cond)
                VALUES (home_player_id, match_row.home_team_api_id, match_row.league_id, match_row.id, i, 'home');
            END IF;

            EXECUTE format(
                'SELECT %I FROM match WHERE id = %L',
                format('away_player_%s', i),
                match_row.id
            ) INTO away_player_id;

            IF away_player_id IS NOT NULL THEN
                INSERT INTO match_player (player_id, team_id, league_id, match_id, position, cond)
                VALUES (away_player_id, match_row.away_team_api_id, match_row.league_id, match_row.id, i, 'away');
            END IF;
        END LOOP;
    END LOOP;
END;
$$ LANGUAGE plpgsql;

++
||
++
++

#### Here we execute our function as this is the way to do it in PostgreSQL

In [17]:
%%sql psql
SELECT
  populate_match_player ();

populate_match_player
""


#### Here we show the table to review if it is right

In [18]:
%%sql psql
SELECT
  *
FROM
  match_player
LIMIT
  10;

id,player_id,team_id,match_id,league_id,position,cond
1,39890,9996,145,1,1,home
2,34480,8635,145,1,1,away
3,38388,8635,145,1,2,away
4,38788,9996,145,1,3,home
5,26458,8635,145,1,3,away
6,38312,9996,145,1,4,home
7,13423,8635,145,1,4,away
8,26235,9996,145,1,5,home
9,38389,8635,145,1,5,away
10,38798,8635,145,1,6,away


#### 2. SQL Queries

Consider the new `match_player` table above and write SQL queries for each of the following information needs.

1. List all the teams that 'Cristiano Ronaldo' has played for (regardless of starting position)

In [19]:
%%sql psql
SELECT DISTINCT
  c.team_long_name
FROM
  match_player a
  LEFT JOIN player b ON a.player_id = b.player_api_id
  LEFT JOIN team c ON a.team_id = c.team_api_id
WHERE
  b.player_name = 'Cristiano Ronaldo';

team_long_name
Manchester United
Real Madrid CF


2. Who are the players that have played all the away games for their team? Output the `team_long_name` and `player_name`.

In [20]:
%%sql psql
SELECT
  c.team_long_name,
  d.player_name
FROM
  (
    SELECT
      player_id,
      team_id,
      COUNT(*) AS count_ma
    FROM
      match_player
    WHERE
      cond = 'away'
    GROUP BY
      player_id,
      team_id
  ) a
  LEFT JOIN (
    SELECT
      away_team_api_id,
      COUNT(*) AS count_matches_away
    FROM
      MATCH
    GROUP BY
      away_team_api_id
  ) b ON a.team_id = b.away_team_api_id
  LEFT JOIN team c ON a.team_id = c.team_api_id
  LEFT JOIN player d ON a.player_id = d.player_api_id
WHERE
  a.count_ma = b.count_matches_away;

team_long_name,player_name
CD Tenerife,Sergio Aragones
DSC Arminia Bielefeld,Dennis Eilhoff
SV Darmstadt 98,Christian Mathenia
Bournemouth,Andrew Surman
Falkirk,Darren Barr
KAS Eupen,Ervin Zukanovic
Karlsruher SC,Sebastian Freis
RC Recreativo,Asier Riesgo
FC Energie Cottbus,Gerhard Tremmel
FC Ingolstadt 04,Marvin Matip


3. What is the match with the highest number of goals? Output `team_long_name` for both home and away teams, and the total number of goals for each team.

In [21]:
%%sql psql
SELECT
  c.team_long_name AS away_team,
  away_team_goal,
  d.team_long_name AS home_team,
  home_team_goal,
  total_goals
FROM
  (
    SELECT
      id,
      away_team_api_id,
      home_team_api_id,
      home_team_goal,
      away_team_goal,
      (home_team_goal + away_team_goal) AS total_goals
    FROM
      MATCH
  ) a
  INNER JOIN (
    SELECT
      MAX(home_team_goal + away_team_goal) AS most_goals_match
    FROM
      MATCH
  ) b ON total_goals = most_goals_match
  LEFT JOIN team c ON c.team_api_id = a.away_team_api_id
  LEFT JOIN team d ON d.team_api_id = a.home_team_api_id;

away_team,away_team_goal,home_team,home_team_goal,total_goals
Hibernian,6,Motherwell,6,12
Rayo Vallecano,2,Real Madrid CF,10,12
